# **COMP529 Assignment2 Part 1. Data Analysis using PySpark**

Lizhenghe.Chen 201521681

I will use Notebook as my part1 report

Can run my code successfully in VM with swiched java version 8 (1.8) and python 2.7


In [36]:
# import necessary package and intialize spark
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql.window import Window

# Can think of it as multithreading
spark = SparkSession.builder.master("local[*]").appName("COMP529").getOrCreate()
path = "dataset.txt"  # the file path


Read txt data using .csv format from spark, also specificate format and data type
you can complre the read file speed between Spark and Pandas, since the dataset is still small: 2537331x8, so the difference isn't obvious, but Spark is definitely faster for big data:Pandas read and print: 2.6s; Spark read and print: 0.3s

In [37]:
import pandas as pd

data = pd.read_csv(path, header=None)
data


C:\Users\15547\AppData\Local\Temp\ipykernel_24044\3908080286.py:3: DtypeWarning: Columns (0,1,2,3,4,5) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv(path, header=None)


,0,1,2,3,4,5,6,7
0,UserID,Latitude,Longitude,AllZero,Altitude,Timestamp,Date,Time
1,100,39.974408918,116.303522101,0,480.287355643045,40753.5306944444,2011-07-29,12:44:12
2,100,39.974397078,116.303526932,0,480.121151574803,40753.5307060185,2011-07-29,12:44:13
3,100,39.973982524,116.303621837,0,478.499455380577,40753.5307291667,2011-07-29,12:44:15
4,100,39.973943291,116.303632641,0,479.176988188976,40753.5307407407,2011-07-29,12:44:16
...,...,...,...,...,...,...,...,...
2537326,130,39.9753,116.330264,0,146.78855,40098.545093,2009-10-12,13:04:56
2537327,130,39.975259,116.330317,0,142.033094,40098.545127,2009-10-12,13:04:59
2537328,130,39.975224,116.330385,0,139.996073,40098.545162,2009-10-12,13:05:02
2537329,130,39.975193,116.330448,0,139.141322,40098.545185,2009-10-12,13:05:04


In [38]:
df = spark.read.option("header", "true").option("delimiter", ",").csv(path)

df = (
    df.withColumn("UserID", df.UserID.cast(IntegerType()))
    .withColumn("Latitude", df.Latitude.cast(DoubleType()))
    .withColumn("Longitude", df.Longitude.cast(DoubleType()))
    .withColumn("Altitude", df.Altitude.cast(DoubleType()))
    .withColumn("Timestamp", df.Timestamp.cast(DoubleType()))
    .withColumn("Date", df.Date.cast(DateType()))
)
df.printSchema()
df.orderBy(col("UserID").desc()).show()


root
 |-- UserID: integer (nullable = true)
 |-- Latitude: double (nullable = true)
 |-- Longitude: double (nullable = true)
 |-- AllZero: string (nullable = true)
 |-- Altitude: double (nullable = true)
 |-- Timestamp: double (nullable = true)
 |-- Date: date (nullable = true)
 |-- Time: string (nullable = true)

+------+---------+----------+-------+----------------+----------------+----------+--------+
|UserID| Latitude| Longitude|AllZero|        Altitude|       Timestamp|      Date|    Time|
+------+---------+----------+-------+----------------+----------------+----------+--------+
|   130|39.975088| 116.33269|      0|492.126135170604|40000.7996759259|2009-07-06|19:11:32|
|   130|39.974945|116.334041|      0|479.549799868766|40000.8007638889|2009-07-06|19:13:06|
|   130| 39.97504|116.332806|      0|491.743412073491|   40000.7996875|2009-07-06|19:11:33|
|   130|39.975009|116.332997|      0|491.676630577428|40000.7996990741|2009-07-06|19:11:34|
|   130|39.975048|116.332932|      0|491

# **1.**

To make all dates and times add 8 hours to Beijing time, my idea is, first combine "Date" and "Time" to DateTime, which include date and time at the same time;

Then convert this new column to timestamp datatype that the computer can recognize date and time. The final step is straightforward: minus 8 hours for "DateTime", the computer will judge and calculate whether the date also needs to change if the time is earlier than 8:00 am.

For the "TimeStamp", as we already know, it is the number of days that have passed since 12/30/1899. So to minuses 8 hours, it means minus 8/24 days.


In [39]:
df_concat = df.withColumn("DateTime", concat_ws(" ", df["Date"], df["Time"]))
# df_concat.show()
newDf = df_concat.withColumn("DateTime", to_timestamp("DateTime"))
newDf.show(10)


+------+------------+-------------+-------+----------------+----------------+----------+--------+-------------------+
|UserID|    Latitude|    Longitude|AllZero|        Altitude|       Timestamp|      Date|    Time|           DateTime|
+------+------------+-------------+-------+----------------+----------------+----------+--------+-------------------+
|   100|39.974408918|116.303522101|      0|480.287355643045|40753.5306944444|2011-07-29|12:44:12|2011-07-29 12:44:12|
|   100|39.974397078|116.303526932|      0|480.121151574803|40753.5307060185|2011-07-29|12:44:13|2011-07-29 12:44:13|
|   100|39.973982524|116.303621837|      0|478.499455380577|40753.5307291667|2011-07-29|12:44:15|2011-07-29 12:44:15|
|   100|39.973943291|116.303632641|      0|479.176988188976|40753.5307407407|2011-07-29|12:44:16|2011-07-29 12:44:16|
|   100|39.973937148|116.303639667|      0|479.129432414698|40753.5307523148|2011-07-29|12:44:17|2011-07-29 12:44:17|
|   100|39.973916715| 116.30363848|      0|479.615278871

In [40]:
DF1 = newDf.withColumn(
    "DateTime", newDf.DateTime + expr("INTERVAL +8 HOURS")
)  # ahead 8 hours
# below, extrat date from 'DateTime' to 'Date', and same for 'Time'
DF1 = (
    DF1.withColumn("Date", to_timestamp("DateTime").cast("date"))
    .withColumn("Timestamp", col("Timestamp") + 8 / 24)
    .withColumn("Time", date_format("DateTime", "HH:mm:ss"))
    .drop("DateTime")
)

print("Old table:")
df.drop("AllZero").show(10)
print("New table with Beijing Time:")
DF1.drop("AllZero").show(10)


Old table:
+------+------------+-------------+----------------+----------------+----------+--------+
|UserID|    Latitude|    Longitude|        Altitude|       Timestamp|      Date|    Time|
+------+------------+-------------+----------------+----------------+----------+--------+
|   100|39.974408918|116.303522101|480.287355643045|40753.5306944444|2011-07-29|12:44:12|
|   100|39.974397078|116.303526932|480.121151574803|40753.5307060185|2011-07-29|12:44:13|
|   100|39.973982524|116.303621837|478.499455380577|40753.5307291667|2011-07-29|12:44:15|
|   100|39.973943291|116.303632641|479.176988188976|40753.5307407407|2011-07-29|12:44:16|
|   100|39.973937148|116.303639667|479.129432414698|40753.5307523148|2011-07-29|12:44:17|
|   100|39.973916715| 116.30363848|479.615278871391|40753.5307638889|2011-07-29|12:44:18|
|   100|39.973892264|116.303644867|480.506026902887| 40753.530775463|2011-07-29|12:44:19|
|   100|39.973867401|116.303647142| 481.38750984252| 40753.530787037|2011-07-29|12:44:20|

# **2.**

First, group by UserID and Date, then sum each group by using count(), this will filter dupicate dates that only keep one day for each user each day


In [41]:
trimdf = df.select("UserID", "Date")
trimdf.printSchema()
# this will count any day with at least one (>=1) data point
question2 = trimdf.groupBy("UserID", "Date").count().orderBy("UserID", "Date")
question2.show(10)
# question2.count()


root
 |-- UserID: integer (nullable = true)
 |-- Date: date (nullable = true)

+------+----------+-----+
|UserID|      Date|count|
+------+----------+-----+
|   100|2011-07-29| 1976|
|   100|2011-07-31|  483|
|   100|2011-08-02|  551|
|   100|2011-08-03|  577|
|   100|2011-08-08|  666|
|   100|2011-08-09| 2004|
|   101|2007-11-30|  307|
|   101|2007-12-02|  232|
|   101|2007-12-03|   56|
|   101|2007-12-07|  159|
+------+----------+-----+
only showing top 10 rows



drop useless 'count'


In [42]:
question2 = question2.drop("count")
question2.show(10)
# question2.count()


+------+----------+
|UserID|      Date|
+------+----------+
|   100|2011-07-29|
|   100|2011-07-31|
|   100|2011-08-02|
|   100|2011-08-03|
|   100|2011-08-08|
|   100|2011-08-09|
|   101|2007-11-30|
|   101|2007-12-02|
|   101|2007-12-03|
|   101|2007-12-07|
+------+----------+
only showing top 10 rows



Then group by "UserID", sum all dates for each UserID;

sort desc() will let the biggest one at first


In [43]:
df2 = question2.groupBy("UserID").count()  # goup by the same ID

windowDept = Window.partitionBy("count").orderBy(
    col("count").desc(), col("UserID").asc()
)
# in case of a tie, output the user with the smaller ID
df2 = (
    df2.withColumn("row", row_number().over(windowDept))
    .filter(col("row") == 1)
    .drop("row")
)

df2.orderBy(col("count").desc()).show(40)


print("Top 5 user with number of days that has recorded:")
print("there are %d units in total" % df2.count())
df2.orderBy(col("count").desc()).show(5)


+------+-----+
|UserID|count|
+------+-----+
|   128|  909|
|   126|  181|
|   115|  123|
|   112|  108|
|   104|  106|
|   125|   50|
|   119|   44|
|   101|   40|
|   111|   39|
|   102|   29|
|   103|   22|
|   110|   19|
|   113|   18|
|   114|   16|
|   129|    9|
|   105|    8|
|   108|    7|
|   100|    6|
|   121|    5|
|   118|    4|
|   106|    3|
|   120|    2|
+------+-----+

Top 5 user with number of days that has recorded:
there are 22 units in total
+------+-----+
|UserID|count|
+------+-----+
|   128|  909|
|   126|  181|
|   115|  123|
|   112|  108|
|   104|  106|
+------+-----+
only showing top 5 rows



# **3.**


we can see UserID with count 31 on 2008-08-09 will be removed


In [44]:
# first remove the dublicated dates
newdf = trimdf.groupBy("UserID", "Date").count().orderBy("UserID")
newdf.show(10)
# filter any day which more than 100 data points recorded
question3 = newdf.filter("count >=100")
question3.show(10)


+------+----------+-----+
|UserID|      Date|count|
+------+----------+-----+
|   100|2011-07-29| 1976|
|   100|2011-08-03|  577|
|   100|2011-07-31|  483|
|   100|2011-08-08|  666|
|   100|2011-08-02|  551|
|   100|2011-08-09| 2004|
|   101|2008-04-07|  313|
|   101|2008-05-22|  565|
|   101|2008-05-09|   31|
|   101|2008-03-24|  123|
+------+----------+-----+
only showing top 10 rows

+------+----------+-----+
|UserID|      Date|count|
+------+----------+-----+
|   100|2011-08-02|  551|
|   100|2011-08-03|  577|
|   100|2011-07-31|  483|
|   100|2011-08-08|  666|
|   100|2011-07-29| 1976|
|   100|2011-08-09| 2004|
|   101|2008-06-29| 2469|
|   101|2007-12-07|  159|
|   101|2008-02-02|  612|
|   101|2007-12-15|  112|
+------+----------+-----+
only showing top 10 rows



In [45]:
question3 = question3.groupBy("UserID").count()  # goup and by the same ID
question3 = question3.withColumnRenamed("count", "number of days")
size = question3.count()

print("all user IDs that with number of days that at least 100 data points per day:")
question3.sort(col("number of days").desc()).show(size)
print("there are %d units in total" % size)


all user IDs that with number of days that at least 100 data points per day:
+------+--------------+
|UserID|number of days|
+------+--------------+
|   128|           875|
|   126|           175|
|   115|           110|
|   112|            98|
|   104|            77|
|   125|            47|
|   119|            41|
|   101|            31|
|   102|            26|
|   103|            22|
|   111|            21|
|   130|            19|
|   122|            18|
|   113|            18|
|   114|            15|
|   110|            13|
|   105|             8|
|   124|             7|
|   127|             7|
|   100|             6|
|   121|             5|
|   123|             4|
|   106|             3|
|   116|             3|
|   129|             3|
|   118|             3|
|   108|             2|
|   120|             2|
|   109|             2|
+------+--------------+

there are 29 units in total


# **4.**


first select useful colum, and make sure the data types are correct


In [46]:
altitudedf = df.select("UserID", "Altitude", "Date")
altitudedf.show()
altitudedf.printSchema()


+------+----------------+----------+
|UserID|        Altitude|      Date|
+------+----------------+----------+
|   100|480.287355643045|2011-07-29|
|   100|480.121151574803|2011-07-29|
|   100|478.499455380577|2011-07-29|
|   100|479.176988188976|2011-07-29|
|   100|479.129432414698|2011-07-29|
|   100|479.615278871391|2011-07-29|
|   100|480.506026902887|2011-07-29|
|   100| 481.38750984252|2011-07-29|
|   100|482.008727034121|2011-07-29|
|   100|482.325816929134|2011-07-29|
|   100|482.289422572178|2011-07-29|
|   100|482.314353674541|2011-07-29|
|   100|482.362713254593|2011-07-29|
|   100|482.472345800525|2011-07-29|
|   100|482.716797900262|2011-07-29|
|   100|482.782529527559|2011-07-29|
|   100|483.086404199475|2011-07-29|
|   100|483.396486220472|2011-07-29|
|   100|483.624166666667|2011-07-29|
|   100|484.271414041995|2011-07-29|
+------+----------------+----------+
only showing top 20 rows

root
 |-- UserID: integer (nullable = true)
 |-- Altitude: double (nullable = true)
 |

- first, Partition the DataFrame on "UserID" column, which groups all same departments into a group.

- Apply orderBy() on "Altitude" column by descending order, "Date" for ascending to make sure output the earliest day if tie.
- Add a new column row by running row_number() function over the partition window. row_number() function returns a sequential number starting from 1 within a window partition group.
- Using the PySpark filter(), just select row == 1, which returns the maximum "Altitude" of each group.
- Finally, if a row column is not needed, just drop it.


In [47]:
# https://sparkbyexamples.com/pyspark/pyspark-find-maximum-row-per-group-in-dataframe/
# col("Date").asc() means in case of a tie, output the earliest such a day
windowDept = Window.partitionBy("UserID").orderBy(
    col("Altitude").desc(), col("Date").asc()
)
question4 = (
    altitudedf.withColumn("row", row_number().over(windowDept))
    .filter(col("row") == 1)
    .drop("row")
)
print("Top 5 user with the highest altitude:")
question4.orderBy(col("Altitude").desc()).show(5)


Top 5 user with the highest altitude:
+------+----------------+----------+
|UserID|        Altitude|      Date|
+------+----------------+----------+
|   128|        107503.3|2009-11-02|
|   106|36581.3648293963|2007-10-09|
|   103|         25259.2|2008-09-12|
|   101|         24806.4|2008-03-28|
|   126|         19432.4|2008-06-22|
+------+----------------+----------+
only showing top 5 rows



# **5.**


In [48]:
trimdf = df.select("UserID", "Timestamp")
# trimdf.show()
# make sure the data type are correct
question5 = trimdf.withColumn("Timestamp", trimdf.Timestamp.cast(DoubleType()))
# question5.printSchema()

# qaq=question5.groupBy('UserID','Timestamp').count()
question5 = question5.sort(col("UserID").asc())
# question5.show()
# question5.count()


Same principle as question4, this time also include minimum, the only difference is order byTimestamp ascend or descendly


In [49]:
windowDept = Window.partitionBy("UserID").orderBy(col("Timestamp").asc())
min = (
    question5.withColumn("row", row_number().over(windowDept))
    .filter(col("row") == 1)
    .drop("row")
    .withColumnRenamed("Timestamp", "Min_Timestamp")
)
print("UserID with Minimum TimeStamp list:")
min.show(10)

windowDept = Window.partitionBy("UserID").orderBy(col("Timestamp").desc())
max = (
    question5.withColumn("row", row_number().over(windowDept))
    .filter(col("row") == 1)
    .drop("row")
    .withColumnRenamed("Timestamp", "Max_Timestamp")
)
print("UserID with Maximum TimeStamp list:")
max.show(10)


UserID with Minimum TimeStamp list:
+------+----------------+
|UserID|   Min_Timestamp|
+------+----------------+
|   100|40753.5306944444|
|   101|39416.6085300926|
|   102|40833.5650810185|
|   103|39687.2470023148|
|   104|39416.6075347222|
|   105|39357.0945833333|
|   106|39363.0810763889|
|   107|39357.3149189815|
|   108|39357.6008680556|
|   109|39416.6085300926|
+------+----------------+
only showing top 10 rows

UserID with Maximum TimeStamp list:
+------+----------------+
|UserID|   Max_Timestamp|
+------+----------------+
|   100|40764.6604861111|
|   101|39629.3044791667|
|   102|40908.6375810185|
|   103|39711.4569097222|
|   104|39733.8255092593|
|   105|39365.4873842593|
|   106|39365.5583449074|
|   107|39362.0612962963|
|   108|39364.0281481481|
|   109|39419.6934027778|
+------+----------------+
only showing top 10 rows



join the minimum and maximum timestamp to one dataframe and prepare to be calculated


In [50]:
question5 = max.join(min, max.UserID == min.UserID).drop(min.UserID)
# caclulate the time span for each user
question5 = question5.withColumn(
    "Timespan", (question5["Max_Timestamp"] - question5["Min_Timestamp"])
)
question5 = question5.drop("Max_Timestamp", "Min_Timestamp")
question5.show(10)


+------+------------------+
|UserID|          Timespan|
+------+------------------+
|   100|11.129791666695382|
|   101| 212.6959490740992|
|   102| 75.07249999999476|
|   103| 24.20990740739944|
|   104|317.21797453710315|
|   105| 8.392800925998017|
|   106| 2.477268518494384|
|   107| 4.746377314797428|
|   108|6.4272800925027695|
|   109|3.0848726851982065|
+------+------------------+
only showing top 10 rows



In [51]:
print("Top 5 highest Timespan UserID:")
question5.sort(col("Timespan").desc()).show(5)
# print("there are %d units in total" % question5.count())


Top 5 highest Timespan UserID:
+------+-----------------+
|UserID|         Timespan|
+------+-----------------+
|   128|1426.294375000005|
|   114|963.8455902778005|
|   111|838.7832175925942|
|   115|506.6880439814995|
|   126| 325.831643518497|
+------+-----------------+
only showing top 5 rows



# **6.**


firstly, create a new frame that only contain neccessary colums


In [52]:
trimdf = df.select("UserID", "Latitude", "Longitude", "Date", "Timestamp").orderBy(
    col("UserID").asc(), col("Timestamp").asc()
)
trimdf.show(10)
# trimdf.printSchema()


+------+------------+-------------+----------+----------------+
|UserID|    Latitude|    Longitude|      Date|       Timestamp|
+------+------------+-------------+----------+----------------+
|   100|39.974408918|116.303522101|2011-07-29|40753.5306944444|
|   100|39.974397078|116.303526932|2011-07-29|40753.5307060185|
|   100|39.973982524|116.303621837|2011-07-29|40753.5307291667|
|   100|39.973943291|116.303632641|2011-07-29|40753.5307407407|
|   100|39.973937148|116.303639667|2011-07-29|40753.5307523148|
|   100|39.973916715| 116.30363848|2011-07-29|40753.5307638889|
|   100|39.973892264|116.303644867|2011-07-29| 40753.530775463|
|   100|39.973867401|116.303647142|2011-07-29| 40753.530787037|
|   100|39.973836462| 116.30365019|2011-07-29|40753.5307986111|
|   100|39.973821199|116.303649412|2011-07-29|40753.5308101852|
+------+------------+-------------+----------+----------------+
only showing top 10 rows



create a window below, and then lag each Latitude and Longitude one row, partition by UserID, so each UserID's first row shoud be null.


In [53]:
windowSpec = Window.partitionBy("UserID").orderBy(col("Timestamp").asc())
df_lag = trimdf.withColumn(
    "prev_Latitude", lag(trimdf["Latitude"]).over(windowSpec)
).withColumn("prev_Longitude", lag(trimdf["Longitude"]).over(windowSpec))
# df_lag=df_lag.na.fill(value=0,subset=["prev_Latitude","prev_Longitude"])
df_lag.filter(col("UserID") == 100).show(10)
# df_lag.printSchema()


+------+------------+-------------+----------+----------------+-------------+--------------+
|UserID|    Latitude|    Longitude|      Date|       Timestamp|prev_Latitude|prev_Longitude|
+------+------------+-------------+----------+----------------+-------------+--------------+
|   100|39.974408918|116.303522101|2011-07-29|40753.5306944444|         null|          null|
|   100|39.974397078|116.303526932|2011-07-29|40753.5307060185| 39.974408918| 116.303522101|
|   100|39.973982524|116.303621837|2011-07-29|40753.5307291667| 39.974397078| 116.303526932|
|   100|39.973943291|116.303632641|2011-07-29|40753.5307407407| 39.973982524| 116.303621837|
|   100|39.973937148|116.303639667|2011-07-29|40753.5307523148| 39.973943291| 116.303632641|
|   100|39.973916715| 116.30363848|2011-07-29|40753.5307638889| 39.973937148| 116.303639667|
|   100|39.973892264|116.303644867|2011-07-29| 40753.530775463| 39.973916715|  116.30363848|
|   100|39.973867401|116.303647142|2011-07-29| 40753.530787037| 39.973

To express sin, cos and radian function, I will need sql function form pyspark,

the caculation method is from: https://stackoverflow.com/questions/19412462/getting-distance-between-two-points-based-on-latitude-longitude


In [54]:
import pyspark.sql.functions as F

distance_list = (
    df_lag.withColumn(
        "a",
        F.sin((F.radians(col("Latitude")) - F.radians(col("prev_Latitude"))) / 2) ** 2
        + F.cos((F.radians(col("prev_Latitude"))))
        * F.cos((F.radians(col("Latitude"))))
        * F.sin((F.radians(col("Longitude")) - F.radians(col("prev_Longitude"))) / 2)
        ** 2,
    )
    .withColumn(
        "distance(meter)",
        2 * F.atan2(F.sqrt(F.col("a")), F.sqrt(1 - F.col("a"))) * 6373.0 * 1000,
    )
    .drop("a")
)
# show some of rows to check
distance_list.filter(col("UserID") == 100).drop(
    "TimeStamp", "prev_Latitude", "prev_Longitude"
).show(10)


+------+------------+-------------+----------+------------------+
|UserID|    Latitude|    Longitude|      Date|   distance(meter)|
+------+------------+-------------+----------+------------------+
|   100|39.974408918|116.303522101|2011-07-29|              null|
|   100|39.974397078|116.303526932|2011-07-29|1.3798395802400303|
|   100|39.973982524|116.303621837|2011-07-29| 46.81501218227461|
|   100|39.973943291|116.303632641|2011-07-29| 4.459995088445842|
|   100|39.973937148|116.303639667|2011-07-29|0.9085982359806887|
|   100|39.973916715| 116.30363848|2011-07-29|2.2750102315565335|
|   100|39.973892264|116.303644867|2011-07-29|2.7736368609033817|
|   100|39.973867401|116.303647142|2011-07-29| 2.772297938081956|
|   100|39.973836462| 116.30365019|2011-07-29|  3.45113330739803|
|   100|39.973821199|116.303649412|2011-07-29|1.6989956937999133|
+------+------------+-------------+----------+------------------+
only showing top 10 rows



checking computations for the first value where UserID = 100:
As we can see the first row from distance_list is almost the same as mpu caculated, the are both around 1.379 meters


In [55]:
# %pip install mpu
import mpu

lat1 = 39.974408918
lon1 = 116.303522101
lat2 = 39.974397078
lon2 = 116.303526932

mpu.haversine_distance((lat1, lon1), (lat2, lon2)) * 1000  # km->m


1.3794065535071975

group by UserID and sum each group's distance(meter)


In [56]:
question6 = (
    distance_list.groupBy("UserID", "Date")
    .sum("distance(meter)")
    .drop("TimeStamp", "prev_Latitude", "prev_Longitude")
).orderBy("UserID", col("sum(distance(meter))").desc())
question6.show()


+------+----------+--------------------+
|UserID|      Date|sum(distance(meter))|
+------+----------+--------------------+
|   100|2011-07-29|   10968.19726137189|
|   100|2011-08-09|   9859.201742002968|
|   100|2011-08-03|  3007.3359004551553|
|   100|2011-08-08|  2992.7149981476996|
|   100|2011-07-31|  2914.2650684240202|
|   100|2011-08-02|  2322.4813446678218|
|   101|2008-01-25|   912636.5609156062|
|   101|2008-03-02|   910255.5372850713|
|   101|2007-12-22|  222879.25910870914|
|   101|2008-02-02|  189791.11993210213|
|   101|2008-03-09|   185714.6327218315|
|   101|2007-12-19|  157989.98987679632|
|   101|2007-12-15|  134268.30041232018|
|   101|2007-12-13|   131311.6597017672|
|   101|2008-03-12|  114941.11641117312|
|   101|2008-03-30|   97868.81779553965|
|   101|2008-03-28|    90380.8490922154|
|   101|2007-12-08|   73516.50969499162|
|   101|2008-06-29|   39708.42500351011|
|   101|2007-11-30|   35724.77945028216|
+------+----------+--------------------+
only showing top

For each user output the (earliest) day they travelled the most, So sort by both Date(asc for earliest) and distance(desc for largest), same method using window as question 4 and 5,

as we can see the largest one is User 128, who is also the longest collecting time user


In [57]:
windowDept = Window.partitionBy("UserID").orderBy(
    col("sum(distance(meter))").desc(), col("Date").asc()
)  # make sure it is also earliest day in each first row
max = (
    question6.withColumn("row", row_number().over(windowDept))
    .filter(col("row") == 1)
    .drop("row")
    .withColumnRenamed("sum(distance(meter))", "Max_Distance(meter)")
)

size = max.count()
print("The (earliest) day each User travelled the most: ")
max.orderBy(col("Max_Distance(meter)").desc()).show(size)
print("there are %d units in total" % size)


The (earliest) day each User travelled the most: 
+------+----------+--------------------+
|UserID|      Date| Max_Distance(meter)|
+------+----------+--------------------+
|   128|2009-02-22|1.0093183885299848E7|
|   124|2008-10-03|   4277820.620757491|
|   111|2007-09-05|   2462793.930192698|
|   115|2007-11-28|  2098104.3645122917|
|   122|2009-07-21|   2005624.137404955|
|   125|2008-08-27|  1597832.1468177768|
|   112|2008-02-02|  1078721.9733951034|
|   127|2008-10-05|  1028823.4643552401|
|   123|2009-09-23|   964125.4511769849|
|   101|2008-01-25|   912636.5609156062|
|   120|2009-09-19|  464563.23009168444|
|   126|2008-05-01|  372628.64234369097|
|   129|2008-05-02|   317812.7642064925|
|   106|2007-10-08|  269624.04448305944|
|   118|2007-05-20|  223659.21346841284|
|   103|2008-09-12|  176983.89397442676|
|   110|2008-01-18|   159683.2755984129|
|   119|2008-09-02|  156787.47962323824|
|   108|2007-10-04|  147051.66030393587|
|   121|2009-10-08|  135596.80158893982|
|   104

Finally, just simply sum all sumed users to get the total distance, divided by 1000 to turn metric to kilometric


In [58]:
question6.select(sum("sum(distance(meter))")).withColumn(
    "Total Distance(km)", col("sum(sum(distance(meter)))") / 1000
).drop("sum(sum(distance(meter)))").show()


+------------------+
|Total Distance(km)|
+------------------+
| 180859.0390624603|
+------------------+

